In [2]:
from mlxtend.data import loadlocal_mnist
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import csv

C:\Users\sshek\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
with open('../name.csv', mode='w') as name_file:
    name_writer = csv.writer(name_file, delimiter=',')
    name_writer.writerow(['sshekha4'])

In [4]:
train_val_images, train_val_labels = loadlocal_mnist(
        images_path='../dataset/train-images.idx3-ubyte', 
        labels_path='../dataset/train-labels.idx1-ubyte')
test_images, test_labels = loadlocal_mnist(
        images_path='../dataset/t10k-images.idx3-ubyte', 
        labels_path='../dataset/t10k-labels.idx1-ubyte')

In [23]:
print('Dimensions: %s x %s' % (train_val_images.shape[0], train_val_images.shape[1]))
print(train_val_labels[1])

Dimensions: 60000 x 784
0


In [5]:
# Dividing the training set into training and validation set for parameter tuning
train_images, val_images, train_labels, val_labels = train_test_split(train_val_images, train_val_labels, test_size=0.2, random_state=1)

In [6]:
## Logistic Regression

# Varying the value of C to find the best value
C = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
max_score = 0.0
for val in C:
    logisticRegr = LogisticRegression(solver="lbfgs", C=val)
    logisticRegr.fit(train_images, train_labels)
    score = logisticRegr.score(val_images, val_labels)
    if(score >= max_score):
        max_score = score
        C_opt = val
    print("Accuracy = {0}% for C = {1}".format(score*100, val))
print("Max Accuracy = {0}% for C = {1}".format(max_score*100, C_opt))

Accuracy = 91.31666666666666% for C = 0.001
Accuracy = 91.19166666666668% for C = 0.01
Accuracy = 91.24166666666666% for C = 0.1
Accuracy = 91.21666666666667% for C = 1
Accuracy = 91.2% for C = 10
Accuracy = 91.21666666666667% for C = 100
Accuracy = 91.18333333333334% for C = 1000
Max Accuracy = 91.31666666666666% for C = 0.001


In [7]:
# Training the model using the value of C that gives highest score with validation data
logisticRegr = LogisticRegression(solver="lbfgs", C=C_opt)
logisticRegr.fit(train_val_images, train_val_labels)

LogisticRegression(C=0.001, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
          verbose=0, warm_start=False)

In [8]:
# Prediction using Logistic Regression
predictions = logisticRegr.predict(test_images)
print(predictions)

[7 2 1 ... 4 5 6]


In [9]:
#Creating a dataframe of 10000*10 with zeros
zero_data = np.zeros(shape=(10000,10), dtype='int32')
df = pd.DataFrame(data = zero_data, index = pd.Series(range(1,10001)))

In [10]:
#Modifying the Dataframe
for row,col in df.iterrows():
    df.at[row, predictions[row-1]] = 1
#Saving the dataframe to csv file
df.to_csv("../lr.csv", index=False, header=False)

In [11]:
#Calculating the accuracy of the Model
score = logisticRegr.score(test_images, test_labels)
print("Accuracy =", score*100, "%")

Accuracy = 91.75999999999999 %


In [12]:
## Random Forest

# Varying the value of n_estmators to find the best value
# Varying the value of max_depth to find the best value
n_estimators = [5, 10, 20, 50]
max_depth = [5, 10, 25, 50]
max_score = 0.0
# Using Grid search to find the optimal value
for val1 in n_estimators:
    for val2 in max_depth:
        randForest = RandomForestClassifier(n_estimators=val1, max_depth=val2, random_state=0)
        randForest.fit(train_images, train_labels)
        score = randForest.score(val_images, val_labels)
        if(score >= max_score):
            max_score = score
            n_estimators_opt = val1
            max_depth_opt = val2
        print("Accuracy = {0}% for n_estimators = {1} and max_depth = {2}".format(score*100, val1, val2))
print("Max Accuracy = {0}% for n_estimators = {1} and max_depth = {2}".format(max_score*100, n_estimators_opt, max_depth_opt))

Accuracy = 75.7% for n_estimators = 5 and max_depth = 5
Accuracy = 89.5% for n_estimators = 5 and max_depth = 10
Accuracy = 90.91666666666667% for n_estimators = 5 and max_depth = 25
Accuracy = 91.19166666666668% for n_estimators = 5 and max_depth = 50
Accuracy = 79.44166666666666% for n_estimators = 10 and max_depth = 5
Accuracy = 92.04166666666667% for n_estimators = 10 and max_depth = 10
Accuracy = 94.30833333333334% for n_estimators = 10 and max_depth = 25
Accuracy = 94.19166666666666% for n_estimators = 10 and max_depth = 50
Accuracy = 82.85% for n_estimators = 20 and max_depth = 5
Accuracy = 93.48333333333333% for n_estimators = 20 and max_depth = 10
Accuracy = 95.81666666666668% for n_estimators = 20 and max_depth = 25
Accuracy = 95.66666666666667% for n_estimators = 20 and max_depth = 50
Accuracy = 84.36666666666667% for n_estimators = 50 and max_depth = 5
Accuracy = 93.95833333333333% for n_estimators = 50 and max_depth = 10
Accuracy = 96.68333333333334% for n_estimators = 50 

In [13]:
# Training the model using the value of C that gives highest score with validation data
randForest = RandomForestClassifier(n_estimators=n_estimators_opt, max_depth=max_depth_opt, random_state=0)
randForest.fit(train_val_images, train_val_labels)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=25, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [14]:
# Prediction using Logistic Regression
predictions = randForest.predict(test_images)
print(predictions)

[7 2 1 ... 4 5 6]


In [15]:
#Calculating the accuracy of the Model
score = randForest.score(test_images, test_labels)
print("Accuracy =", score*100, "%")

Accuracy = 96.71 %


In [16]:
#Creating a dataframe of 10000*10 with zeros
zero_data = np.zeros(shape=(10000,10), dtype='int32')
df = pd.DataFrame(data = zero_data, index = pd.Series(range(1,10001)))

In [17]:
#Modifying the Dataframe
for row,col in df.iterrows():
    df.at[row, predictions[row-1]] = 1
#Saving the dataframe to csv file
df.to_csv("../rf.csv", index=False, header=False)